In [ ]:
# !pip install kaggle
# from google.colab import files
# files.upload()
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle competitions download -c titanic
# !ls

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
misstrain = train.isnull().sum()
misstrain = misstrain[misstrain>0]
misstest = test.isnull().sum()
misstest = misstest[misstest>0]

In [ ]:
train = train.drop(['Cabin','PassengerId','Name'],axis=1)
train['Age'] = train['Age'].fillna(train['Age'].median())#Age is a right skewness distribution
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

In [ ]:
test = test.drop(['Cabin','PassengerId','Name'],axis=1)
test['Fare'] = test['Fare'].fillna(test['Fare'].mode()[0])
test['Age'] = test['Age'].fillna(test['Age'].median())

In [ ]:
qualifeatures = [i for i in train.columns if train.dtypes[i] == 'object']
print(qualifeatures)

['Sex', 'Ticket', 'Embarked']


In [ ]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,1,1,female,35.0,1,0,113803,53.1000,S
4,0,3,male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,S
887,1,1,female,19.0,0,0,112053,30.0000,S
888,0,3,female,28.0,1,2,W./C. 6607,23.4500,S
889,1,1,male,26.0,0,0,111369,30.0000,C


In [ ]:
for i in qualifeatures:
  train[i] = pd.factorize(train[i])[0]

In [ ]:
for i in qualifeatures:
  test[i] = pd.factorize(test[i])[0]

In [ ]:
cor = train.corr()
train_cor = cor['Survived'].to_dict()
print(train_cor)

{'Survived': 1.0, 'Pclass': -0.33848103596101586, 'Sex': 0.5433513806577526, 'Age': -0.06491041993052575, 'SibSp': -0.03532249888573588, 'Parch': 0.08162940708348222, 'Ticket': -0.0472976189186144, 'Fare': 0.2573065223849618, 'Embarked': 0.10681138570892197}


In [ ]:
import_features = []
for key, value in train_cor.items():
  if ((value >= 0.05) and (value < 0.9)) or (value <= -0.05):
    import_features.append(key)
print(import_features)

['Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'Embarked']


In [ ]:
final_train = pd.DataFrame(train).drop(['Survived'],axis=1)
final_test = pd.DataFrame(test)
log_train = pd.DataFrame()
log_test = pd.DataFrame()


for i in import_features:
  log_train[i] = train[i]
  log_test[i] = test[i]

In [ ]:
log_train

,Pclass,Sex,Age,Parch,Fare,Embarked
0,3,0,22.0,0,7.2500,0
1,1,1,38.0,0,71.2833,1
2,3,1,26.0,0,7.9250,0
3,1,1,35.0,0,53.1000,0
4,3,0,35.0,0,8.0500,0
...,...,...,...,...,...,...
886,2,0,27.0,0,13.0000,0
887,1,1,19.0,0,30.0000,0
888,3,1,28.0,2,23.4500,0
889,1,0,26.0,0,30.0000,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
scale = preprocessing.StandardScaler().fit(final_train)
std_train = scale.transform(final_train)
std_test = scale.transform(final_test)
y = train['Survived']

In [ ]:
scalelog = preprocessing.StandardScaler().fit(log_train)
std_log_train = scalelog.transform(log_train)
std_log_test = scalelog.transform(log_test)
y = train['Survived']

In [ ]:
rf = RandomForestClassifier(oob_score=True, random_state=28, n_estimators=100, max_features=None)
rf.fit(std_train, y)
print(rf.oob_score_)


0.8204264870931538


In [ ]:
train_x,val_x,train_y,val_y = train_test_split(std_train,y,train_size = 0.7, random_state = 28)

In [ ]:
rf_alltrain_y = rf.predict(std_train)

print('-'*10 + ' Random Forest ' + '-'*10)
print('R Square for alltrain', r2_score(y, rf_alltrain_y))
print('MSE',mean_squared_error(y, rf_alltrain_y))
print('RMSE',mean_squared_error(y, rf_alltrain_y, squared=False))

---------- Random Forest ----------
R Square for alltrain 0.9905090595340811
MSE 0.002244668911335578
RMSE 0.04737793696791343


In [ ]:
rf_train_y = rf.predict(train_x)

print('-'*10 + ' Random Forest ' + '-'*10)
print('R Square for train', r2_score(train_y, rf_train_y))
print('MSE',mean_squared_error(train_y, rf_train_y))
print('RMSE',mean_squared_error(train_y, rf_train_y, squared=False))

---------- Random Forest ----------
R Square for train 0.9932117329149233
MSE 0.0016051364365971107
RMSE 0.04006415401075019


In [ ]:
rf_val_y = rf.predict(val_x)

print('-'*10 + ' Random Forest ' + '-'*10)
print('R Square for val', r2_score(val_y, rf_val_y))
print('MSE',mean_squared_error(val_y, rf_val_y))
print('RMSE',mean_squared_error(val_y, rf_val_y, squared=False))

---------- Random Forest ----------
R Square for val 0.9842306560753162
MSE 0.0037313432835820895
RMSE 0.06108472217815261


In [ ]:
train1_x,val1_x,train1_y,val1_y = train_test_split(std_log_train,y,train_size = 0.8, random_state = 28)

In [ ]:
logireg = LogisticRegression()
logireg.fit(train1_x,train1_y)
log_train_y = logireg.predict(train1_x)
log_val_y = logireg.predict(val1_x)

print('-'*10 + ' Logistic ' + '-'*10)
print('R Square for train', r2_score(train1_y, log_train_y))
print('R square for Accuracy', r2_score(val1_y, log_val_y))
print('MSE',mean_squared_error(val1_y, log_val_y))
print('RMSE',mean_squared_error(val1_y, log_val_y, squared=False))

---------- Logistic ----------
R Square for train 0.1384146139309752
R square for Accuracy 0.14877717391304324
MSE 0.19553072625698323
RMSE 0.4421885641408914


In [ ]:
import math

In [ ]:
# Predict = rf.predict(std_test)

In [ ]:
Predict = np.round(0.5*rf.predict(std_test)+0.5*logireg.predict(std_log_test)).astype(int)

In [ ]:
Predict

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# Predict = (0.5*rf.predict(std_test)+0.5*logireg.predict(std_test))

In [ ]:
id  = pd.read_csv("test.csv")['PassengerId']

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = id
submission['Survived'] = Predict
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
submission.to_csv('Titanic.csv',index=False)
from google.colab import files
files.download('Titanic.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>